# 1. 패키지 설치

In [6]:
%pip install langsmith openai python-dotenv langchain langchain-openai langchain-pinecone bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [bs4]
Note: you may need to restart the kernel to use updated packages.


# 2. 데이터 생성

- Evaluation에 활용될 Question - Answer pair 생성

# LLM Evaluation
할루시네이션 예방
잘못된 답변을 방지하기 위한 최소한의 도구
LangSmith를 활용한 Evaluation

## Dataset 만들기


In [13]:
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["OPENAI_API_KEY"] = "up_FeRaFreJQmiZyVXY5cNGqUwWisws2"

In [14]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_upstage import UpstageEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# List of URLs to load documents from
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

# Load documents from the URLs
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Initialize a text splitter with specified chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)

# Split the documents into chunks
doc_splits = text_splitter.split_documents(docs_list)

# Add the document chunks to the "vector store" using OpenAIEmbeddings
vectorstore = InMemoryVectorStore.from_documents(
    documents=doc_splits,
    embedding=UpstageEmbeddings(model="solar-embedding-1-large"),
)

# With langchain we can easily turn any vector store into a retrieval component:
retriever = vectorstore.as_retriever(k=6)

In [15]:
from langchain_upstage import ChatUpstage
from langsmith import traceable

llm = ChatUpstage()

# Add decorator so this function is traced in LangSmith
@traceable()
def rag_bot(question: str) -> dict:
    # LangChain retriever will be automatically traced
    docs = retriever.invoke(question)
    docs_string = "".join(doc.page_content for doc in docs)

    instructions = f"""You are a helpful assistant who is good at analyzing source information and answering questions.       Use the following source documents to answer the user's questions.       If you don't know the answer, just say that you don't know.       Use three sentences maximum and keep the answer concise.

Documents:
{docs_string}"""

    # langchain ChatModel will be automatically traced
    ai_msg = llm.invoke([
            {"role": "system", "content": instructions},
            {"role": "user", "content": question},
        ],
    )

    return {"answer": ai_msg.content, "documents": docs}

In [16]:
from langsmith import Client

client = Client()

# Define the examples for the dataset
examples = [
    {
        "inputs": {"question": "제1조에 따른 소득세법의 목적은 무엇인가요?"},
        "outputs": {"answer": "소득세법의 목적은 소득의 성격과 납세자의 부담능력에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지하는 것입니다."},
    },
    {
        "inputs": {"question": "'거주자'는 소득세법에서 어떻게 정의되나요?"},
        "outputs": {"answer": "'거주자'는 한국에 주소를 두거나 183일 이상 거소를 둔 개인을 의미합니다."},
    },
    {
        "inputs": {"question": "'비거주자'는 소득세법에 따라 어떻게 정의되나요?"},
        "outputs": {"answer": "'비거주자'는 거주자가 아닌 개인을 의미합니다."},
    },
    {
        "inputs": {"question": "소득세법에 따른 '내국법인'은 누구를 의미하나요?"},
        "outputs": {"answer": "'내국법인'은 법인세법 제2조 제1호에 따른 내국법인을 의미합니다."},
    },
    {
        "inputs": {"question": "소득세법에 따라 소득세를 납부할 의무가 있는 사람은 누구인가요?"},
        "outputs": {"answer": "거주자 및 국내원천소득이 있는 비거주자는 소득세를 납부할 의무가 있습니다."},
    },
    {
        "inputs": {"question": "거주자의 과세 범위는 무엇인가요?"},
        "outputs": {"answer": "거주자는 법에서 규정한 모든 소득에 대해 과세되며, 비거주자는 국내원천소득에 대해서만 과세됩니다."},
    },
    {
        "inputs": {"question": "소득세법에 따라 소득은 어떻게 분류되나요?"},
        "outputs": {"answer": "소득은 종합소득, 퇴직소득, 양도소득으로 분류됩니다."},
    },
    {
        "inputs": {"question": "종합소득이란 무엇인가요?"},
        "outputs": {"answer": "종합소득은 이자소득, 배당소득, 사업소득, 근로소득, 연금소득 및 기타소득을 포함합니다."},
    },
    {
        "inputs": {"question": "세금이 면제되는 소득의 종류는 무엇인가요?"},
        "outputs": {"answer": "비과세 소득에는 공익신탁의 이익, 특정 사업소득 및 기타 법에서 정한 특정 소득이 포함됩니다."},
    },
    {
        "inputs": {"question": "소득세의 과세기간은 어떻게 되나요?"},
        "outputs": {"answer": "소득세의 과세기간은 매년 1월 1일부터 12월 31일까지입니다."},
    },
    {
        "inputs": {"question": "거주자의 소득세 납세지는 어디인가요?"},
        "outputs": {"answer": "거주자의 소득세 납세지는 주소지이며, 주소지가 없으면 거소지입니다."},
    },
    {
        "inputs": {"question": "비거주자의 소득세 납세지는 어디인가요?"},
        "outputs": {"answer": "비거주자의 소득세 납세지는 국내사업장의 소재지입니다. 국내사업장이 여러 곳인 경우 주된 사업장의 소재지가 납세지가 됩니다."},
    },
    {
        "inputs": {"question": "납세지가 불분명한 경우 어떻게 되나요?"},
        "outputs": {"answer": "납세지가 불분명한 경우 대통령령으로 정합니다."},
    },
    {
        "inputs": {"question": "원천징수세액의 납세지는 어떻게 결정되나요?"},
        "outputs": {"answer": "원천징수세액의 납세지는 원천징수자의 종류와 위치에 따라 결정됩니다."},
    },
    {
        "inputs": {"question": "납세자의 사망 시 납세지는 어떻게 되나요?"},
        "outputs": {"answer": "납세자의 사망 시 상속인 또는 납세관리인의 주소지나 거소지가 납세지가 됩니다."},
    },
    {
        "inputs": {"question": "신탁 소득에 대한 납세의 범위는 무엇인가요?"},
        "outputs": {"answer": "신탁 소득에 대한 납세의 범위는 신탁의 수익자가 해당 소득에 대해 납세의무를 집니다."},
    },
    {
        "inputs": {"question": "원천징수 대상 소득은 무엇인가요?"},
        "outputs": {"answer": "이자소득, 배당소득 및 기타 법에서 정한 소득은 원천징수 대상입니다."},
    },
    {
        "inputs": {"question": "공동 소유 자산의 양도소득은 어떻게 과세되나요?"},
        "outputs": {"answer": "공동 소유 자산의 양도소득은 각 거주자 소유 지분에 따라 과세됩니다."},
    },
    {
        "inputs": {"question": "이자 소득의 출처는 무엇인가요?"},
        "outputs": {"answer": "이자 소득의 출처는 정부 및 지방자치단체가 발행한 채권, 법인이 발행한 채권, 국내외 은행 예금 등입니다."},
    },
    {
        "inputs": {"question": "소득세법에서 배당소득은 어떻게 정의되나요?"},
        "outputs": {"answer": "배당소득은 국내외 법인으로부터 받는 배당금 및 배분금, 기타 법에서 정한 소득을 포함합니다."},
    },
]

# Create the dataset and examples in LangSmith
dataset_name = "income_tax_dataset"
dataset = client.create_dataset(dataset_name=dataset_name)
client.create_examples(
    dataset_id=dataset.id,
    examples=examples
)

/Users/jhlee/.pyenv/versions/3.10.17/envs/streamlit/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


LangSmithAuthError: Authentication failed for /datasets. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/datasets', '{"detail":"Invalid token"}')

# 3. Retriever 생성

- Evaluation을 위한 Retriever 생성
- 3.x 에서 생성한 Pinecone Retriever 

In [17]:
from dotenv import load_dotenv

load_dotenv()

True

In [19]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()

embedding = UpstageEmbeddings(model="solar-embedding-1-large"),

In [20]:
from langchain_pinecone import PineconeVectorStore

index_name = 'table-markdown-index'

database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)
retriever = database.as_retriever()

# 4. LLM 답변 생성을 위한 RagBot 

- retriever와 OpenAI API를 활용한 RAG

In [21]:
### RAG bot

import openai
from langsmith import traceable
from langsmith.wrappers import wrap_openai

class RagBot:

    def __init__(self, retriever, model: str = "gpt-4o"):
        # 위에서 선언한 retriever를 할용해서 Retrieval 실행
        self._retriever = retriever
        # Wrapping the client instruments the LLM
        # LangSmith 문법
        self._client = wrap_openai(openai.Client())
        self._model = model

    @traceable()
    def retrieve_docs(self, question):
        return self._retriever.invoke(question)

    @traceable()
    def invoke_llm(self, question, docs):
        # `retrieve_docs()` 를 통해 가져온 문서들을 system prompt로 전달
        # 3.3에서 했던 방식과 유사함
        response = self._client.chat.completions.create(
            model=self._model,
            messages=[
                {
                    "role": "system",
                    "content": "당신은 한국의 소득세 전문가입니다."
                    "아래 소득세법을 참고해서 사용자의 질문에 답변해주세요.\n\n"
                    f"## 소득세법\n\n{docs}",
                },
                {"role": "user", "content": question},
            ],
        )

        # Evaluators 를 활용해서 `answer`와 `context`를 평가할 예정
        return {
            "answer": response.choices[0].message.content,
            "contexts": [str(doc) for doc in docs],
        }

    @traceable()
    def get_answer(self, question: str):
        docs = self.retrieve_docs(question)
        return self.invoke_llm(question, docs)

rag_bot = RagBot(retriever)

In [22]:
def predict_rag_answer(example: dict):
    """답변만 평가할 때 사용"""
    response = rag_bot.get_answer(example["input_question"])
    return {"answer": response["answer"]}

def predict_rag_answer_with_context(example: dict):
    """Context를 활용해서 hallucination을 평가할 때 사용"""
    response = rag_bot.get_answer(example["input_question"])
    return {"answer": response["answer"], "contexts": response["contexts"]}

In [23]:
from langchain import hub
from langchain_upstage import ChatUpstage

# Grade prompt
# 답변의 정확도를 측정하기위해 사용되는 프롬프트
grade_prompt_answer_accuracy = prompt = hub.pull("langchain-ai/rag-answer-vs-reference")

def answer_evaluator(run, example) -> dict:
    """
    RAG 답변 성능을 측정하기 위한 evaluator
    """

    # `example`이 데이터를 생성할 때 입력한 `Question-Answer` pair. `run`은 `RagBot`을 활용해서 생성한 LLM의 답변
    input_question = example.inputs["input_question"]
    reference = example.outputs["output_answer"]
    prediction = run.outputs["answer"]

    # LLM Judge로 사용될 LLM
    llm = ChatUpstage()

    # LLM 응답을 위한 LCEL 활용
    # 3.6 `dictionary_chain`의 `prompt | llm | StrOutputParser()`` 의 구조와 유사함
    answer_grader = grade_prompt_answer_accuracy | llm

    # Evaluator 실행
    score = answer_grader.invoke({"question": input_question,
                                  "correct_answer": reference,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_v_reference_score", "score": score}

/Users/jhlee/.pyenv/versions/3.10.17/envs/streamlit/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [24]:
# Grade prompt
# 답변이 사용자의 질문에 얼마나 도움되는지 판단하는 프롬프트
grade_prompt_answer_helpfulness = prompt = hub.pull("langchain-ai/rag-answer-helpfulness")

def answer_helpfulness_evaluator(run, example) -> dict:
    """
    답변이 사용자의 질문에 얼마나 도움되는지 판단하는 Evaluator
    """

    # 데이터셋의 답변과 비교하지 않고, 데이터셋의 질문에 대한 LLM의 답변의 가치를 평가함
    input_question = example.inputs["input_question"]
    prediction = run.outputs["answer"]

    # LLM Judge로 사용될 LLM
    llm = ChatUpstage()

    # LLM 응답을 위한 LCEL 활용
    # 3.6 `dictionary_chain`의 `prompt | llm | StrOutputParser()`` 의 구조와 유사함
    answer_grader = grade_prompt_answer_helpfulness | llm

    # Evaluator 실행
    score = answer_grader.invoke({"question": input_question,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_helpfulness_score", "score": score}

/Users/jhlee/.pyenv/versions/3.10.17/envs/streamlit/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [25]:
# Prompt
# hallucination 판단을 위한 프롬프트
grade_prompt_hallucinations = prompt = hub.pull("langchain-ai/rag-answer-hallucination")

def answer_hallucination_evaluator(run, example) -> dict:
    """
    hallucination 판단을 위한 Evaluator
    """

    # 데이터셋에 있는 질문과, LLM이 답변을 생성할 때 사용한 context를 활용
    input_question = example.inputs["input_question"]
    contexts = run.outputs["contexts"]

    # LLM의 답변
    prediction = run.outputs["answer"]

    # LLM Judge로 사용될 LLM
    llm = ChatUpstage()

    # LLM 응답을 위한 LCEL 활용
    # 3.6 `dictionary_chain`의 `prompt | llm | StrOutputParser()`` 의 구조와 유사함
    answer_grader = grade_prompt_hallucinations | llm

    # Evaluator 실행
    score = answer_grader.invoke({"documents": contexts,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_hallucination", "score": score}

/Users/jhlee/.pyenv/versions/3.10.17/envs/streamlit/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [29]:
from langsmith.evaluation import RunEvaluator

# RunEvaluator 초기화
evaluator = RunEvaluator(
    model="gpt-4o",  # 사용할 모델 지정
    dataset="income_tax_dataset",  # 평가에 사용할 데이터셋 이름
    evaluators=["accuracy", "helpfulness", "hallucination"],  # 실행할 평가 항목
    metadata={"version": "income tax v1, gpt-4o"}  # 메타데이터 추가
)

# 평가 실행
experiment_results = evaluator.evaluate(
    predict_function=predict_rag_answer_with_context,  # 평가에 사용할 함수
    experiment_prefix="inflearn-evaluator-lecture-hallucination"  # 실험 이름
)

# 평가 결과 출력
print("Evaluation Results:", experiment_results)

TypeError: RunEvaluator() takes no arguments